In [159]:
import json
from typing import Any

In [160]:
# set up data provenance

dir = '/home/nathan/Documents/nomad-FAIR/dependencies/parsers/electronic/electronicparsers/fhiaims/'
tier_ranking = {'light': 3, 'tight': 2, 'really_tight': 1}  # ranked by descending hierarchy
tiers = sorted(tier_ranking, key=tier_ranking.get)


In [161]:
def extract_build_info(parserInfo: dict[str, Any]) -> dict[str, str]:
    """
    Return the relevant subset of parserInfo
    """
    new_parser_info: dict[str, str] = {}
    new_parser_info['parser_name'] = parserInfo['name']
    for tag in ['version', 'sbtVersion', 'scalaVersion']:
        new_parser_info[tag] = parserInfo['versionInfo'][tag]
    return new_parser_info

In [162]:
def filter_native_quantities(elements: dict[str, Any]) -> dict[str, Any]:
    """
    Filter out all non-application quantities from the elements dictionary.
    """
    filtered_elements: dict[str, Any] = {}
    for tag, element in elements.items():
        # distinguish application quantities
        if tag[:2] == 'x_':
            if isinstance(element, list):
                if all(isinstance(x, dict) for x in element):
                    filtered_elements[tag] = [filter_native_quantities(spec) for spec in element]
                    continue
            filtered_elements[tag] = element
    return filtered_elements

In [163]:
def replace_indexed_quantities(elements: dict[str, Any]) -> dict[str, Any]:
    """
    Replace indexed quantities with a list of comparable entries.
    This implementation assumes that all indexes are present.
    """
    filtered_elements: dict[str, Any] = {}
    new_quantities: dict[str, dict[Any, int]] = {}
    numbers = [str(i) for i in range(10)]
    for tag, element in elements.items():
        # distinguish indexed quantities
        if tag[-1] in numbers:
            new_tag, indexed = tag[:-1], int(tag[-1])
            # store indexed quantities separately in `new_quantities`
            # importantly, they aren't stored in `filtered_elements` directly
            if new_tag not in new_quantities:
                new_quantities[new_tag] = {}
            if isinstance(element, list):
                element = tuple(element)
            new_quantities[new_tag][element] = indexed
        else:
            filtered_elements[tag] = element
    for new_tag, new_quantity in new_quantities.items():
        # add sorted indexed quantities
        filtered_elements[new_tag] = sorted(new_quantity.keys(), key=lambda x: new_quantity[x])
    return filtered_elements

In [164]:
def filter_quantity_name(element: dict[str, Any], quantity_names: list[str]) -> dict[str, Any]:
    """
    Filter out the name of the quantity from the element dictionary.
    """
    filtered_element: dict[str, Any] = {}
    for tag, value in element.items():
        if tag not in quantity_names:
            filtered_element[tag] = value
    return filtered_element

In [165]:
# read in and process all data

filtered_quantities = [
    'x_fhi_aims_controlIn_nucleus',
    'x_fhi_aims_controlIn_mass',
    'x_fhi_aims_controlIn_species_name',
    'x_fhi_aims_controlIn_angular_grids_method',
]  # filter out other identifiers

all_tiers: dict[dict[str, dict[str, Any]]] = {'data': {}}
read_info = True
for tier_name in tiers:
    filename = f'{tier_name}.json'
    with open(filename, 'r') as f:
        tier_data = json.load(f)

        # extract build info
        if read_info:
            all_tiers['info'] = extract_build_info(tier_data['parserInfo'])
            read_info = False

        # extract reference data defining tier
        data = tier_data['sections']['section_run-0']['sections']['section_method-0']['x_fhi_aims_section_controlIn_basis_set']
        storage = all_tiers['data']
        for element in data:
            element_name = element['x_fhi_aims_controlIn_species_name']
            if element_name not in storage:
                storage[element_name] = {}
            storage[element_name][f'{tier_name}_2020'] =\
                replace_indexed_quantities(
                    filter_quantity_name(
                        filter_native_quantities(element),
                        filtered_quantities
                    )
                )

In [166]:
# sample test the final result

print(all_tiers['info'])
all_tiers['data']['H']['light_2020']

{'parser_name': 'FhiAimsControlInParser', 'version': '2.0.0-6-g84af92a', 'sbtVersion': '0.13.13', 'scalaVersion': '2.11.8'}


{'x_fhi_aims_controlIn_radial_multiplier': 1,
 'x_fhi_aims_controlIn_outer_grid': 302.0,
 'x_fhi_aims_controlIn_l_hartree': 4,
 'x_fhi_aims_controlIn_basis_dep_cutoff': 0.0001,
 'x_fhi_aims_section_controlIn_basis_func': [{'x_fhi_aims_controlIn_basis_func_l': 's',
   'x_fhi_aims_controlIn_basis_func_radius': 1.0,
   'x_fhi_aims_controlIn_basis_func_type': 'valence',
   'x_fhi_aims_controlIn_basis_func_n': 1},
  {'x_fhi_aims_controlIn_basis_func_l': 's',
   'x_fhi_aims_controlIn_basis_func_radius': 0.5,
   'x_fhi_aims_controlIn_basis_func_type': 'ion_occ',
   'x_fhi_aims_controlIn_basis_func_n': 1},
  {'x_fhi_aims_controlIn_basis_func_l': 's',
   'x_fhi_aims_controlIn_basis_func_radius': 2.1,
   'x_fhi_aims_controlIn_basis_func_type': 'hydro',
   'x_fhi_aims_controlIn_basis_func_n': 2},
  {'x_fhi_aims_controlIn_basis_func_l': 'p',
   'x_fhi_aims_controlIn_basis_func_radius': 3.5,
   'x_fhi_aims_controlIn_basis_func_type': 'hydro',
   'x_fhi_aims_controlIn_basis_func_n': 2}],
 'x_fhi_aim

In [167]:
# write the data to disk
with open('native_tier_references.json', 'w') as f:
    json.dump(all_tiers, f, indent=4)
    f.close()